In [1]:
!pip install torch-geometric


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid
from sklearn.model_selection import train_test_split

In [5]:
# Load dataset
dataset = Planetoid(root='data/Planetoid/Cora', name='Cora')
data = dataset[0]
x_train, x_test, y_train, y_test = train_test_split(data.x, data.y, test_size=0.2, random_state=42)
train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
train_mask[:len(y_train)] = True
test_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
test_mask[len(y_train):] = True
data.train_mask = train_mask
data.test_mask = test_mask

In [4]:
data.x

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [5]:
# Define GCN model
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        #self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.conv4 = GCNConv(hidden_channels, out_channels)
        
    def forward(self, x, edge_index):
        x = F.dropout(x, training=self.training)
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.conv2(x, edge_index))
        #x = F.dropout(x, training=self.training)
       # x = F.relu(self.conv3(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv4(x, edge_index)
        return F.log_softmax(x, dim=1)

In [6]:
model = GCN(dataset.num_features, 16, dataset.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [57]:
# Train the model
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    print(f"loss:{loss}, epoch:{epoch}")
    loss.backward()
    optimizer.step()

loss:1.9590909481048584, epoch:0
loss:1.924805998802185, epoch:1
loss:1.8863941431045532, epoch:2
loss:1.8399745225906372, epoch:3
loss:1.782397747039795, epoch:4
loss:1.721042275428772, epoch:5
loss:1.6498091220855713, epoch:6
loss:1.6115037202835083, epoch:7
loss:1.5579825639724731, epoch:8
loss:1.503568172454834, epoch:9
loss:1.4150114059448242, epoch:10
loss:1.3586270809173584, epoch:11
loss:1.29390549659729, epoch:12
loss:1.1971880197525024, epoch:13
loss:1.1451987028121948, epoch:14
loss:1.1326372623443604, epoch:15
loss:1.0444974899291992, epoch:16
loss:0.9689077734947205, epoch:17
loss:0.9190168380737305, epoch:18
loss:0.918247640132904, epoch:19
loss:0.8865355253219604, epoch:20
loss:0.8240237236022949, epoch:21
loss:0.7778937220573425, epoch:22
loss:0.760765790939331, epoch:23
loss:0.7068468332290649, epoch:24
loss:0.685701310634613, epoch:25
loss:0.6652517318725586, epoch:26
loss:0.6715267896652222, epoch:27
loss:0.6326756477355957, epoch:28
loss:0.61064213514328, epoch:29
l

In [58]:
# Evaluate the model
model.eval()
out = model(data.x, data.edge_index)
pred = out.argmax(dim=1)
train_acc = pred[data.train_mask].eq(data.y[data.train_mask]).sum().item() / data.train_mask.sum().item()
test_acc = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item() / data.test_mask.sum().item()
print(f'GCN Train Accuracy: {train_acc:.4f}')
print(f'GCN Test Accuracy: {test_acc:.4f}')

GCN Train Accuracy: 0.9488
GCN Test Accuracy: 0.8801


# GAT

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class GAT(nn.Module):
    def __init__(self, in_channels, out_channels, heads):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, out_channels, heads=heads)
        self.conv2 = GATConv(out_channels*heads, dataset.num_classes, heads=1)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)
     


c:\g\pproject\aimbot\bot\bot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
data.num_features

1433

In [8]:
model = GAT(1433, 8,2)
# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = torch.nn.CrossEntropyLoss()

In [6]:
# Train the model
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    print(f"loss:{loss}, epoch:{epoch}")
    loss.backward()
    optimizer.step()

loss:1.978661060333252, epoch:0
loss:1.8766239881515503, epoch:1
loss:1.7958424091339111, epoch:2
loss:1.7161864042282104, epoch:3
loss:1.6727346181869507, epoch:4
loss:1.5920881032943726, epoch:5
loss:1.5551594495773315, epoch:6
loss:1.4943780899047852, epoch:7
loss:1.4494061470031738, epoch:8
loss:1.3956613540649414, epoch:9
loss:1.2998195886611938, epoch:10
loss:1.287372350692749, epoch:11
loss:1.257125735282898, epoch:12
loss:1.1703777313232422, epoch:13
loss:1.1107853651046753, epoch:14
loss:1.0667933225631714, epoch:15
loss:1.0587174892425537, epoch:16
loss:0.9956143498420715, epoch:17
loss:0.956100583076477, epoch:18
loss:0.9193140864372253, epoch:19
loss:0.8778000473976135, epoch:20
loss:0.8770712018013, epoch:21
loss:0.8773687481880188, epoch:22
loss:0.7918104529380798, epoch:23
loss:0.8252235651016235, epoch:24
loss:0.7789773344993591, epoch:25
loss:0.7552055716514587, epoch:26
loss:0.7412589192390442, epoch:27
loss:0.7209650874137878, epoch:28
loss:0.7134753465652466, epoch:

In [7]:
# Evaluate the model
model.eval()
out = model(data.x, data.edge_index)
pred = out.argmax(dim=1)
train_acc = pred[data.train_mask].eq(data.y[data.train_mask]).sum().item() / data.train_mask.sum().item()
test_acc = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item() / data.test_mask.sum().item()
print(f'GAT Train Accuracy: {train_acc:.4f}')
print(f'GAT Test Accuracy: {test_acc:.4f}')

GAT Train Accuracy: 0.9524
GAT Test Accuracy: 0.8911


# GraphSAGE

In [49]:
from torch_geometric.nn import SAGEConv
import torch.nn.functional as F
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [8]:
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

In [50]:
model = GraphSAGE(data.num_features, 16, dataset.num_classes)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

In [51]:
# Train the model
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    print(f"loss:{loss}, epoch:{epoch}")
    loss.backward()
    optimizer.step()

loss:1.944828987121582, epoch:0
loss:1.8258439302444458, epoch:1
loss:1.6843223571777344, epoch:2
loss:1.5395017862319946, epoch:3
loss:1.410620093345642, epoch:4
loss:1.2808737754821777, epoch:5
loss:1.179602861404419, epoch:6
loss:1.0790797472000122, epoch:7
loss:0.9595229029655457, epoch:8
loss:0.8850168585777283, epoch:9
loss:0.8162871599197388, epoch:10
loss:0.7404764294624329, epoch:11
loss:0.7127943634986877, epoch:12
loss:0.6344779133796692, epoch:13
loss:0.5983304977416992, epoch:14
loss:0.5429476499557495, epoch:15
loss:0.5133098363876343, epoch:16
loss:0.503345251083374, epoch:17
loss:0.46643102169036865, epoch:18
loss:0.47617536783218384, epoch:19
loss:0.44016045331954956, epoch:20
loss:0.44639092683792114, epoch:21
loss:0.4048590660095215, epoch:22
loss:0.38690662384033203, epoch:23
loss:0.37629085779190063, epoch:24
loss:0.36286306381225586, epoch:25
loss:0.3608657121658325, epoch:26
loss:0.3434101939201355, epoch:27
loss:0.3152034878730774, epoch:28
loss:0.32910060882568

In [52]:
# Evaluate the model
model.eval()
out = model(data.x, data.edge_index)
pred = out.argmax(dim=1)
train_acc = pred[data.train_mask].eq(data.y[data.train_mask]).sum().item() / data.train_mask.sum().item()
test_acc = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item() / data.test_mask.sum().item()
print(f'GAT Train Accuracy: {train_acc:.4f}')
print(f'GAT Test Accuracy: {test_acc:.4f}')

GAT Train Accuracy: 0.9991
GAT Test Accuracy: 0.8911


In [ ]:
import torch
from torch_geometric.nn import SAGEConv
import torch.nn.functional as F
from torch_geometric.data import DataLoader

class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_channels, hidden_channels)
        self.conv2 = SAGEConv(hidden_channels, 1)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.5, training=self.training)
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return torch.sigmoid(x)

# Assuming your graphs are stored in a list called graph
loader = DataLoader(graph, batch_size=32, shuffle=True)

model = GraphSAGE(dataset.num_node_features, 16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCELoss()

def train():
    model.train()

    for data in loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index)  # Perform a single forward pass.
         loss = criterion(out[data.train_mask], data.y[data.train_mask])  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

for epoch in range(100):
    train()

In [ ]:
for g in graph:
    g.x = torch.tensor([[data['area'], data['intensity']] for _, data in g.nodes(data=True)], dtype=torch.float)
    g.y = torch.tensor([data['lable'] for _, data in g.nodes(data=True)], dtype=torch.float)
    g.edge_attr = torch.tensor([data['weight'] for _, _, data in g.edges(data=True)], dtype=torch.float)